In [ ]:
import networkx as nx
import numpy as np
from itertools import chain,combinations
import itertools
import pandas as pd
import sys
from tqdm.notebook import tqdm, trange
import time

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [ ]:
def delete_edges_from(graph, vertex):
    e = graph.adj[vertex]
    for linked in list(e):
        graph.remove_edge(vertex,linked)

In [ ]:
def sort_by_metric_func(graph):
    return list(graph.nodes())

In [ ]:
def get_metric_ranking(rankings, num):
    return rankings.index(num)

In [ ]:
# Eqn 2 in diversification paper
def calc_compdist_from(graph, i, j, lam):
    
    dimension_dist = 0
    i_props = graph.nodes[i]
    j_props = graph.nodes[j]
    ranking_i = i_props['m0']
    ranking_j = j_props['m0']
    #print("ranking_i" + str(ranking_i))
    #print("ranking_j" + str(ranking_j))
    
    # Get the properties
    for k,v in i_props.items():
        # Dimension values only
        if k.startswith("d"):
            dist = (v - j_props[k])
            dimension_dist += (dist ** 2.0)
            
    comp_value = ranking_i + ranking_j + 2 * lam * np.sqrt(dimension_dist)
    #print("dimension_dist = " + str(dimension_dist))
    #print("comp_value = " + str(comp_value))
    return comp_value
    

In [ ]:
def find_max_weight_edge(sim_graph, lam):
    max_weight = -1
    tu = 0
    tv = 0
    vertex_count = sim_graph.number_of_nodes()
    for u in range(0,vertex_count):
        linked_nodes = sim_graph.adj[u]
        for v in list(linked_nodes):
            w = calc_compdist_from(sim_graph, u, v, lam)
            #print("u="+str(u) + "," + str(v) + ",weight=" + str(w))
            if (w > max_weight):
                tu = u
                tv = v
                max_weight = w
    return tu,tv, max_weight

In [ ]:
def get_total_weight_selected_nodes(simgraph, nodes,lam):
    total_weight = 0
    for i in nodes:
        for j in nodes:
            total_weight += calc_compdist_from(simgraph, i, j, lam)
            
    return total_weight

In [ ]:
a = nx.Graph()
a.add_node(5)
a.add_node(7)
a.add_node(4)
a.nodes

In [ ]:
# Need to add arbitrary vertex
def max_sum_divergence_approx(sim_graph, phy_count, lam):
    Phy = nx.Graph() 
    lim = int(np.floor(phy_count/2))
    for i in range(0,lim):
        mu, mv, weight = find_max_weight_edge(sim_graph, lam)
        if (mu != mv):
            print("Max weight: " + str(mu) + "->" + str(mv) + ",weight=" + str(weight))
            Phy.add_node(mu)
            Phy.add_node(mv)
            Phy.add_edge(mu,mv)
            print("Number of nodes in Phy=" + str(Phy.number_of_nodes()))
            # remove other connected edges from mu and mv in the sim_graph
            delete_edges_from(sim_graph, mu)
            delete_edges_from(sim_graph, mv)
            # check further
        
    # If phy_count is odd, add a random vertex
    # for now, only use an even phy_count
    
    total_weight = get_total_weight_selected_nodes(sim_graph, Phy.nodes ,lam)
    return Phy

In [ ]:
def gen_node_props(metrics_count, dim_count, metric_max, dim_max):
    nodeprops = {}
    metrics = []
    
    for di in range(0, dim_count):
        rand_dim = np.random.uniform(low=0, high=dim_max)
        nodeprops["d" + str(di)] = rand_dim
        
    # Generate the metrics
    for mi in range(0,metrics_count):
        rand_metric = np.random.uniform(low=0, high=metric_max)
        metrics.append(rand_metric)
        nodeprops["m" + str(mi)] = rand_metric
    return nodeprops

In [ ]:
def create_random_sim_graph(node_count, metric_count, metric_max, dim_count, dim_max):
    sim = nx.Graph();
    for i in range(0,node_count):
        sim.add_nodes_from([(i, gen_node_props(metric_count, dim_count, metric_max, dim_max))])
        
    for i in range(0, node_count):
        for j in range(0, node_count):
            if (i != j):
                sim.add_edge(i,j)

    print("Edges: = " + str(sim.number_of_edges()))
    return sim

In [ ]:
def read_node_props_from_dataframe(data, i, chosen_dimensions):
    # map the selected columns in chosen_dimensions into 'd0'
    dim_num = 0
    node_props = {}
    for d in chosen_dimensions:
        dim_id = "d" + str(dim_num)
        node_props[dim_id] = data.loc[i, d]
        dim_num += 1
    node_props['m0'] = data.loc[i, "METRIC_Q"]
    node_props['NAME'] = data.loc[i, "NAME"]
    
    # Set flag if it is on the pareto front
    node_props["TESTTAG"] = data.loc[i, "TESTTAG"]        
    return node_props

In [ ]:
def create_sim_graph_from_data(filename_csv, chosen_dimensions):
    sim = nx.Graph();
    data = pd.read_csv(filename_csv)
    node_count = len(data)
    for i in range(0,node_count):
        sim.add_nodes_from([(i, read_node_props_from_dataframe(data, i, chosen_dimensions))])
        
    for i in range(0, node_count):
        for j in range(0, node_count):
            if (i != j):
                sim.add_edge(i,j)
    return sim

In [ ]:
def extend_dim_boundaries_if_needed(boundaries, dim_id, value):
    if dim_id in boundaries:
        interval = boundaries[dim_id]
        if not (value in interval):
            if value < interval.left:
                boundaries[dim_id] = pd.Interval(value, interval.right)
            if value > interval.right:
                boundaries[dim_id] = pd.Interval(interval.left, value)
    else:
        boundaries[dim_id] = pd.Interval(value,value, 'both')

In [ ]:
# Get the maximum and minimum of particular dimensions
def find_dimensional_boundaries(simgraph):
    boundaries = {}
    node_count = len(simgraph.nodes)
    for i in range(0, node_count):
        for k,v in simgraph.nodes[i].items():
            if k.startswith("d"):
                extend_dim_boundaries_if_needed(boundaries, k, v)
    return boundaries

In [ ]:
def normalise_value(bound_interval,v):
    return (v - bound_interval.left) / bound_interval.length

In [ ]:
def normalise_dimensions(simgraph):
    boundaries = find_dimensional_boundaries(simgraph)
    simgraph_out = nx.Graph()
    node_count = len(simgraph.nodes())
    for i in range(0, node_count):
        new_props = {}
        for k,v in simgraph.nodes[i].items():
            if k.startswith("d"):
                bound_interval = boundaries[k]
                new_props[k] = normalise_value(bound_interval, v)
            else:
                new_props[k] = v
                
        simgraph_out.add_nodes_from([(i, new_props)])
                
    for i in range(0, node_count):
        for j in range(0, node_count):
            if (i != j):
                simgraph_out.add_edge(i,j)
                
    return simgraph_out

In [ ]:
# TODO: simple ranking function is the sum of the metrics
# TODO: metrics would need to be scaled somehow
def ranking_func(graph,node_id):
    props = graph.nodes[node_id]
    total = 0
    for k,v in props.items():
        if "m" in k:
            total += v
    return total

In [ ]:
# dist_sum theoretical max = k * (k-1) * sqrt(param_space_num_dimesions)
# assuming all dimensions are properly normalised from zero to one!

# weight_sum theortical max = k * max_rank

In [ ]:
def bounding_box_all_dimensions_volume(graph, node_set):

    max_dists_by_dim = {}
    max_dist_points = {}

    # Need to override the product to zero when no nodes involved
    if np.size(node_set) > 0:
        product = 1
    else:
        product = 0
    
    for ni in node_set:
        for nj in node_set:
            i_props = graph.nodes[ni]
            j_props = graph.nodes[nj]
            # Dimension values only
            for k,v in i_props.items():
                if k.startswith("d"):
                    dist_dim = np.abs(v - j_props[k])
                    print(k,ni,nj,dist_dim)
                    if (not (k in max_dists_by_dim)):
                        max_dists_by_dim[k] = dist_dim
                    else:
                        if (dist_dim > max_dists_by_dim[k]):
                            max_dists_by_dim[k] = dist_dim
                            max_dist_points[k] = (ni,nj)
                        

    for d in max_dists_by_dim.values():
        product *= d
    
    return (product, max_dists_by_dim, max_dist_points)

In [ ]:
def bounding_box_perimeter_sum(graph, node_set):

    max_dists_by_dim = {}
    max_dist_points = {}

    sum = 0
    
    for ni in node_set:
        for nj in node_set:
            i_props = graph.nodes[ni]
            j_props = graph.nodes[nj]
            # Dimension values only
            for k,v in i_props.items():
                if k.startswith("d"):
                    dist_dim = np.abs(v - j_props[k])
                    print(k,ni,nj,dist_dim)
                    if (not (k in max_dists_by_dim)):
                        max_dists_by_dim[k] = dist_dim
                    else:
                        if (dist_dim > max_dists_by_dim[k]):
                            max_dists_by_dim[k] = dist_dim
                            max_dist_points[k] = (ni,nj)
                        

    for d in max_dists_by_dim.values():
        sum += d
    
    return (sum, max_dists_by_dim, max_dist_points)

In [ ]:
def calc_subset_value_function(graph, dim_count, subset_inds, lam):
    k = len(subset_inds)
    weight_sum = 0
    for ni in subset_inds:
    # ranking_func is w in the equation
        weight_sum += ranking_func(graph, ni)
    
    # Are we double counting the distance here?
    # dist_sum has to be normalised over the number of dimensions?
    # the 2 is to compensate for counting them only once?
    # not needed here?
    dist_sum = 0
    for ni in subset_inds:
        for nj in subset_inds:
            dist_sum += dist_func(graph, ni,nj)
            
    dist_sum_scaled = dist_sum / np.sqrt(dim_count) 
    print(subset_inds, dist_sum_scaled, weight_sum, dist_sum_scaled)
    bounding_box_vol, _, _ = bounding_box_all_dimensions_volume(graph, subset_inds)

    # Removing the 2 factor here, since we are counting across all possible NxN combinations above
    total = (1-lam)*(k-1)*weight_sum + lam*dist_sum_scaled
    return total, weight_sum, dist_sum_scaled, bounding_box_vol

In [ ]:
def dist_func(graph, ni, nj):
    i_props = graph.nodes[ni]
    j_props = graph.nodes[nj]
    dimension_dist = 0.0
    for k,v in i_props.items():
    # Dimension values only
        if k.startswith("d"):
            dist = (v - j_props[k])
            dimension_dist += (dist ** 2.0)
            #print("Adding dist contribution: ", k, v)
    return np.sqrt(dimension_dist)

In [ ]:
def subset_cost(subset):
    return len(subset)

In [ ]:
def meets_cost_constraint(subset, max_cost):
    return (subset_cost(subset) <= max_cost)

In [ ]:
def subsets_viable_sorted_by_cost(simgraph, lam):
    all_subsets = list(powerset(simgraph.nodes()))
    viable_subsets = filter(meets_cost_constraint, all_subsets)
    subset_info = map(lambda subset: (subset, subset_value(simgraph, subset, lam)), viable_subsets)
    return sorted(subset_info, key=lambda info: info[1], reverse=True)

In [ ]:
def best_subset_via_powerset(simgraph, dim_count, lam, max_cost):
    all_subsets = list(powerset(simgraph.nodes()))
    max_val = 0
    best_ranking_sum = 0
    best_dist_sum = 0
    best_subset = None
    best_boundbox_vol = 0
    all_subset_rankings = []
    
    for p in tqdm(all_subsets):
        if meets_cost_constraint(p, max_cost):
            v, ranking_sum, dist_sum, bounding_box_vol = calc_subset_value_function(simgraph, dim_count, p, lam)
            all_subset_rankings.append(p, v, ranking_sum, dist_sum, bounding_box_vol))
            if v > max_val:
                max_val = v
                best_subset = p
                best_ranking_sum = ranking_sum
                best_dist_sum = dist_sum
                best_boundbox_val = bounding_box_vol
    return (best_subset, max_val, best_ranking_sum, best_dist_sum, best_boundbox_val, all_subset_rankings)

In [ ]:
def filter_pareto_only(simgraph):
    simgraph_out = nx.Graph()
    node_count = len(simgraph.nodes())
    i2 = 0;
    for i in range(0, node_count):
        new_props = {}
        props = simgraph.nodes[i]
        if (props['TESTTAG'] == 1):
            for k,v in simgraph.nodes[i].items():
                new_props[k] = v
                new_props['orig_id'] = i
                
            simgraph_out.add_nodes_from([(i2, new_props)])
            i2+=1
                
    #for i in range(0, node_count):
    #    for j in range(0, node_count):
    #        if (i != j):
    #            simgraph_out.add_edge(i,j)
                
    return simgraph_out    

In [ ]:
def get_tests_by_ids(csv_filename,graph,chosen_set):
    df = pd.read_csv(csv_filename)
    output_testnames = map (lambda id: graph.nodes[id]['NAME'], chosen_set)
    # Filter the dataframe by test ID as NAME field
    return df[df['NAME'].isin(output_testnames)]
    

In [ ]:
# Plot the surface and the graph between them
def plot_graph_with_edges_from_subset(g, edge_set, elev, azim, ax_dist, dim_names, filename, dpi=300):
    MAX_MARKER_SIZE = 10
    MIN_MARKER_SIZE = 4
    FONT_LABEL_SIZE=8
    
    x = np.array(list(map(lambda n: g.nodes[n]['d0'], g)))
    y = np.array(list(map(lambda n: g.nodes[n]['d1'], g)))
    z = np.array(list(map(lambda n: g.nodes[n]['d2'], g)))
    # 3d spring layout
    pos = np.vstack((x, y, z)).T
    print("pos\n",pos)
    # Extract node and edge positions from the layout
    node_xyz = np.array([pos[v] for v in g])
    
    node_ranks = np.array(list(map(lambda n: (MIN_MARKER_SIZE + MAX_MARKER_SIZE * ranking_func(g, n))**2.0, g)))
    
    edge_id_set = itertools.combinations(edge_set, r=2)
    #eset_l = list(edge_id_set)
    #print("edge_id_set\n", eset_l)
    edge_xyz = np.array([(pos[t[0]], pos[t[1]]) for t in edge_id_set])
    print("edge_xyz\n",edge_xyz)

    # Create the 3D figure
    fig = plt.figure()
    plt.rc('axes', labelsize=FONT_LABEL_SIZE) 
        
    ax = fig.add_subplot(111, projection="3d")
    ax.axes.set_xlabel(dim_names[0])
    ax.axes.set_ylabel(dim_names[1])
    ax.axes.set_zlabel(dim_names[2])
    #ax.axes.set_xlim3d(left=0, right=1) 
    #ax.axes.set_ylim3d(bottom=0, top=1) 
    #ax.axes.set_zlim3d(bottom=0, top=1) 
    ax.view_init(elev=elev, azim=azim)
    ax.dist = ax_dist

    # Plot the nodes - alpha is scaled by "depth" automatically
    ax.scatter(*node_xyz.T, s=node_ranks, ec="w")

    # Plot the edges
    for vizedge in edge_xyz:
        ax.plot(*vizedge.T, color="tab:gray")
    
    #_format_axes(ax)
    fig.tight_layout()
    plt.savefig(filename, bbox_inches="tight", dpi=dpi)
    plt.show()
    


In [ ]:
plt.rcParams['figure.figsize'] = [8, 3.6]

In [ ]:
# Set the parameters for the subset selection below
# MAX_COST - maximum cost value allowed
# lam - lambda parameter value
# dims - chosen dimensions for the subset selection

In [ ]:
csv_filename="/home/jharbin/eclipse-workspace/PALTesting/models/res/from-server/TestingPAL-coverage-2024_03_03_09_pop32_cutdown_256.modelscenario2-eddi-timebased-faults-coveragereduced-NONDOM-256.csv"

MAX_COST = 5
lam = 1.0
#dims = ['T1_TIME_MIDPOINT_MEAN', 'P1_PARAMETER_MEAN', 'T2_TIME_LENGTH_MEAN']
dims = ['T1_TIME_MIDPOINT_MEAN', 'T2_TIME_LENGTH_MEAN', 'O0_TOTAL_COUNT']

In [ ]:
dim_count = len(dims)
sim = create_sim_graph_from_data(csv_filename, dims)
sim_pareto = filter_pareto_only(sim)
phy, cost, best_ranking, best_distance_sum, best_boundvol, all_rankings = best_subset_via_powerset(sim_pareto, dim_count, lam, MAX_COST)
print("Best selection:" , phy,cost,best_ranking, best_distance_sum)

elev = 50
azim = 120
ax_dist = 13
pdf_filename = "pw-subsetsel-lambda" + str(lam) + ".pdf"
png_filename = "pw-subsetsel-lambda" + str(lam) + ".png"
dpi_png=600
plot_graph_with_edges_from_subset(sim_pareto, phy, elev, azim, ax_dist, dims, pdf_filename)
plot_graph_with_edges_from_subset(sim_pareto, phy, elev, azim, ax_dist, dims, png_filename, dpi_png)

In [ ]:
get_tests_by_ids(csv_filename,sim_pareto,phy)

In [ ]:
all_rankings

In [ ]:
# Max ranking of all - should agree with Best selection ranking when lambda = 0
from operator import itemgetter
max(all_rankings, key=itemgetter(2)) 

In [ ]:
# Max distance sum - should agree when Best selection; max_distance_sum when lambda = 1
from operator import itemgetter
max(all_rankings, key=itemgetter(3)) 

In [ ]:
# Max distance sum - should agree when Best selection; max_distance_sum when lambda = 1
from operator import itemgetter
max(all_rankings, key=itemgetter(4)) 

In [ ]:
sim_pareto.nodes[3]

In [ ]:
bounding_box_all_dimensions(sim_pareto, [1,2,3])